In [33]:
import pandas as pd
import os
from datetime import datetime
import utils.data_utils as data_utils
from collections import defaultdict

LONG_TERM_ACTIONS = ['walking', 'eating', 'smoking', 'discussion']
ACTIONS = data_utils.define_actions('all')
ACTIONS.sort(key=len)

In [34]:
def csv_iter():
    for f in os.listdir('./checkpoint/test'):
        try:
            datetime.strptime(f[-23:-4], "%d-%m-%Y-%H:%M:%S")
            if f.split('.')[-1] == 'csv':
                day = int(f[-23:-21])
                if day > 10:
                    yield f
        except:
            pass

def extend_df(df, long_term=True):
    if long_term:
        actions = LONG_TERM_ACTIONS
        time_lens = [560, 1000]
    else:
        actions = ACTIONS
        time_lens = [80, 160, 320, 400]
    columns_mapping = defaultdict(list)
    for time_len in time_lens:
        for action_name in actions:
            columns_mapping['3d' + str(time_len)].append(action_name + '3d' + str(time_len))
            columns_mapping['usingfulltestset_3d' + str(time_len)].append('usingfulltestset_' + action_name + '3d' + str(time_len))

    for key in columns_mapping:
        df[key] = df[columns_mapping[key]].mean(axis=1)
    
    return list(columns_mapping.keys())

def get_summary_df(merged):
    cols = list(set([i[:-5] for i in merged.columns]))
    new_cols = []
    for col in cols:
        col_name = [c for c in merged.columns if c.startswith(col)]
        new_cols.append(col+'_mean')
        new_cols.append(col+'_std')
        merged[col+'_mean'] = merged[col_name].mean(axis=1)
        merged[col+'_std'] = merged[col_name].std(axis=1)
    return merged[new_cols]

In [38]:
df_lst = []
mini_df_lst = []
for i, f in enumerate(csv_iter()):
    if not f.startswith('5-5-10-10-10-15-15'):
        continue
    print(i, f)
    df = pd.read_csv(os.path.join('checkpoint/test', "./", f))
    df = df.sort_values(by='v_3d')
    df_lst.append(df[extend_df(df, LONG_TERM_ACTIONS)].add_suffix('_run' + str(i)))
    mini_df_lst.append(df[extend_df(df, LONG_TERM_ACTIONS)].head(1).add_suffix('_run' + str(i)).reset_index(drop=True))

merged = pd.concat(mini_df_lst, axis=1)

0 5-5-10-10-10-15-15main_3d_3D_in15_out25_dct_n_30_22-06-2020-16:17:29.csv
2 5-5-10-10-10-15-15main_3d_3D_in15_out25_dct_n_30_23-06-2020-21:18:06.csv
4 5-5-10-10-10-15-15main_3d_3D_in15_out25_dct_n_30_22-06-2020-03:59:13.csv
5 5-5-10-10-10-15-15main_3d_3D_in15_out25_dct_n_30_23-06-2020-04:39:51.csv
9 5-5-10-10-10-15-15main_3d_3D_in15_out25_dct_n_30_21-06-2020-15:45:52.csv


In [39]:
get_summary_df(merged)

,usingfulltestset_3d560_mean,usingfulltestset_3d560_std,usingfulltestset_3d1000_mean,usingfulltestset_3d1000_std,3d1000_mean,3d1000_std,3d560_mean,3d560_std
0,59.312402,0.171246,79.850457,0.280009,66.508968,1.427072,55.718568,1.006279


In [40]:
mini_df_lst[0]

,3d560_run0,usingfulltestset_3d560_run0,3d1000_run0,usingfulltestset_3d1000_run0
0,56.14349,59.327828,66.855033,79.628547


In [41]:
mini_df_lst[1]

,3d560_run2,usingfulltestset_3d560_run2,3d1000_run2,usingfulltestset_3d1000_run2
0,54.224918,59.172015,65.360337,79.474911


In [42]:
mini_df_lst[2]

,3d560_run4,usingfulltestset_3d560_run4,3d1000_run4,usingfulltestset_3d1000_run4
0,56.904558,59.579918,66.913328,80.102856


In [43]:
mini_df_lst[3]

,3d560_run5,usingfulltestset_3d560_run5,3d1000_run5,usingfulltestset_3d1000_run5
0,55.995639,59.329523,68.50588,80.015809


In [44]:
mini_df_lst[4]

,3d560_run9,usingfulltestset_3d560_run9,3d1000_run9,usingfulltestset_3d1000_run9
0,55.324234,59.152728,64.910262,80.030161
